In [1]:
\l p.q

In [2]:
seq:.p.import[`keras;`:models;`:Sequential;*]
keras:.p.import `keras
dense:.p.import[`keras;`:layers;`:Dense;*]
K:.p.import[`keras;`:backend]
pd:.p.import `pandas
np:.p.import `numpy
hvd:.p.import `horovod.tensorflow

/opt/A3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/opt/A3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
hvd[`:init;*][]

{[f;x]embedPy[f;x]}[foreign]enlist


In [4]:
nparray:.p.import[`numpy;`:array;>]

In [5]:
colStr:"IIIII"
d:(colStr;enlist ",")0: `:/home/kkumar/q/l64/hdata.csv

In [6]:
cd:cols d

In [7]:
e:"f"$d`Event;t:"f"$d`Time

In [8]:
x:"f"$flip d cd where not cd in `Event`Time

In [9]:
hvd[`:init;*]
tf:.p.import `tensorflow

k){'[y;x]}/,[enlist]|:@[{[f;x]embedPy[f;x]}].[code[code;]enlist[;;][foreign]]..


In [10]:
ev:tf[`:placeholder;*;pykwargs (`dtype;`name)!(`float64;`e)]
tm:tf[`:placeholder;*;pykwargs (`dtype;`name)!(`float64;`t)]
xv:tf[`:placeholder;*;pykwargs (`dtype;`name)!(`float64;`x)]

In [21]:
trainnet:{[data;ndl;ndn]
    nn:tf[`:layers;`:dense;*;data;3;`activation pykw tf[`:nn;`:relu]];
    i:0;
    while[i<ndl;
        nn:tf[`:layers;`:dense;*;nn;ndn;`activation pykw tf[`:nn;`:relu]];
        i:i+1];
    nn:tf[`:layers;`:dense;*;nn;1;`activation pykw tf[`:nn;`:relu]];
    :nn
    }

In [27]:
fitness:{[lr;ndl;act]
    nn:trainnet[tf[`:convert_to_tensor;*;nparray x til 1000];ndn];
    cost:tf[`:divide;*;tf[`:subtract;*;tf[`:transpose;*;nn];tf[`:log;*;tf[`:cumsum;*;tf[`:exp;*;nn]]]];tf[`:reduce_sum;*;e]];
    opt:tf[`:train;`:GradientDescentOptimizer;*;lr];
    global_step:tf[`:train;`:get_or_create_global_step;*][];
    dop:hvd[`:DistributedOptimizer;*;opt];
    opt:dop[`:minimize;*;cost;`global_step pykw global_step];
    .p.e"import tensorflow as tf";
    .p.e"import horovod.tensorflow as hvd";
    .p.e"config = tf.ConfigProto()";
    .p.e"config.gpu_options.allow_growth=True";
    .p.e"config.gpu_options.visible_device_list=str(hvd.local_rank())";
    config:.p.get`config;
    print config;
    tf[`:add_to_collection;*;`prediction;nn];
    .p.e"l=list()";
    sz:count d;
    mon_sess:tf[`:Session;*;`config pykw config];
    mon_sess[`:run;*;tf[`:global_variables_initializer;*][]];

    i:0;
    .p.set[`mon_sess;mon_sess];
    .p.set[`opt;opt];
    .p.set[`cost;cost];
    .p.set[`x;xv];
    .p.set[`e;ev];
    while[i<sz;
        if[1000<=(sz-i);
            .p.set[`X;nparray x til 1000];
            .p.set[`E;nparray e til 1000];
            o:.p.eval"mon_sess.run([tf.get_collection(\"prediction\"),opt,cost],feed_dict={x:X,e:E})";
            .p.set[`o;o];
            .p.eval"l.append(o[0])"];
        i:i+1000;
        ];
        :1}



In [28]:
dimlr:skopt[`:space;`:Real;*;`low pykw 0.000001;`high pykw 0.01;`prior pykw `log_uniform;`name pykw `learning_rate]
dimndl:skopt[`:space;`:Integer;*;`low pykw 1;`high pykw 5;`name pykw `num_dense_layers];
dimndn:skopt[`:space;`:Integer;*;`low pykw 5;`high pykw 512;`name pykw `num_dense_nodes];


In [29]:
skopt:.p.import `skopt


In [30]:
.p.set[`dimlr;dimlr];
.p.set[`dimndl;dimndl];
.p.set[`dimndn;dimndn]
p)dimensions=list();
p)dimensions.append(dimlr)
p)dimensions.append(dimndl)
p)dimensions.append(dimndn)
dimensions:.p.get`dimensions

In [34]:
/ res:skopt[`:gp_minimize;*;`func pykw fitness;`dimensions pykw dimensions;`acq_func pykw `EI;`n_calls pykw 40]
.p.set[`fitness;fitness];
p)import skopt
p)res = skopt.gp_minimize(func=fitness,dimensions=dimensions,acq_func='EI',n_calls=10)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/A3/lib/python3.6/site-packages/skopt/optimizer/gp.py", line 228, in gp_minimize
    callback=callback, n_jobs=n_jobs)
  File "/opt/A3/lib/python3.6/site-packages/skopt/optimizer/base.py", line 254, in base_minimize
    result = optimizer.tell(next_x, next_y)
  File "/opt/A3/lib/python3.6/site-packages/skopt/optimizer/optimizer.py", line 418, in tell
    self._check_y_is_valid(x, y)
  File "/opt/A3/lib/python3.6/site-packages/skopt/optimizer/optimizer.py", line 560, in _check_y_is_valid
    raise ValueError("`func` should return a scalar")
ValueError: `func` should return a scalar


[0;31mp.c:76 runs pyerr[0m: [0;31mp.c:76 runs pyerr[0m